In [ ]:
import requests
import pandas as pd

In [ ]:
# http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=6f2d4be584aa413f9df230512210306&q=45.53,87.38&date=2008-09-08&tp=1&format=json
# above is example

In [ ]:
#import cleaned NFL schedule
schedule_csv = "NFL_Schedule.csv"

schedule = pd.read_csv(schedule_csv)
schedule.head()

In [ ]:
#import cleaned NFL Stadium Locations
locations_csv = "NFL_Locations.csv"

new_df = pd.read_csv(locations_csv)
new_df

In [ ]:
locations = new_df.drop([new_df.index[3],new_df.index[9],new_df.index[20],new_df.index[19],new_df.index[17],new_df.index[11],new_df.index[12]])
locations = locations.reset_index()
locations = locations[["Franchise","Latitude","Longitude"]]
locations.replace(to_replace="New York Jets", value="New York")
locations.replace(to_replace="Saint Louis", value="St. Louis")

In [ ]:
a = schedule["City"].unique()
print(a)